In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os
sys.path.append(os.path.abspath("/content/drive/MyDrive/Colab Notebooks/ERAV1/S17/"))
os.chdir("/content/drive/MyDrive/Colab Notebooks/ERAV1/S17/")

In [3]:
!ls

BERT	       GPT     pizza_steak_sushi  s17_run.ipynb  transformer.py
download.jpeg  models  __pycache__	  super_repo	 ViT.ipynb


In [4]:
!pip --q install torchinfo
!pip --q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00


In [5]:
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re


In [6]:
from transformer import SentencesDataset, Transformer, get_batch

In [7]:
# bert training


# =============================================================================
# #Init
# =============================================================================
print('bert initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
#pth = 'training.txt'
pth = '/content/drive/MyDrive/Colab Notebooks/ERAV1/S17/BERT/training.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = '/content/drive/MyDrive/Colab Notebooks/ERAV1/S17/BERT/vocab.txt' #'vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)


# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 10000
for it in range(n_iteration):

    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)

    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)

    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_each == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()


# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
path = '/content/drive/MyDrive/Colab Notebooks/ERAV1/S17/BERT/'
np.savetxt(path+'values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open(path+'names.tsv', 'w+').write('\n'.join(s) )


print('end')



bert initializing..
loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...
initializing model...
initializing optimizer and loss...
training...
it: 0  | loss 10.34  | Δw: 1.065
it: 10  | loss 9.67  | Δw: 0.577
it: 20  | loss 9.4  | Δw: 0.363
it: 30  | loss 9.28  | Δw: 0.288
it: 40  | loss 9.12  | Δw: 0.246
it: 50  | loss 8.91  | Δw: 0.216
it: 60  | loss 8.79  | Δw: 0.201
it: 70  | loss 8.62  | Δw: 0.2
it: 80  | loss 8.44  | Δw: 0.188
it: 90  | loss 8.31  | Δw: 0.169
it: 100  | loss 8.11  | Δw: 0.174
it: 110  | loss 8.01  | Δw: 0.167
it: 120  | loss 7.89  | Δw: 0.154
it: 130  | loss 7.73  | Δw: 0.152
it: 140  | loss 7.64  | Δw: 0.139
it: 150  | loss 7.53  | Δw: 0.142
it: 160  | loss 7.39  | Δw: 0.142
it: 170  | loss 7.27  | Δw: 0.136
it: 180  | loss 7.15  | Δw: 0.132
it: 190  | loss 7.01  | Δw: 0.137
it: 200  | loss 6.98  | Δw: 0.136
it: 210  | loss 6.95  | Δw: 0.14
it: 220  | loss 6.9  | Δw: 0.139
it: 230  | loss 6.81  | Δw: 0.138
it: 240  | loss 6.77  | 